In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv("diamonds (1).csv")

In [31]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [32]:
#EDA 

In [33]:
df.isna().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [34]:
df.duplicated().sum()

np.int64(146)

In [35]:
df.drop_duplicates(inplace=True)

In [36]:
df.duplicated().sum()

np.int64(0)

In [37]:
# segregate the feature
X = df.drop('price', axis=1)
y=df['price']
X.shape, y.shape

((53794, 9), (53794,))

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((43035, 9), (10759, 9), (43035,), (10759,))

In [40]:
from sklearn.preprocessing import OrdinalEncoder

In [41]:
cut_order = ['Fair','Good','Very Good', 'Premium', 'Ideal']
color_order = ['D', 'E', 'F', 'G','H', 'I', 'J']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
oe = OrdinalEncoder(categories=[cut_order,color_order,clarity_order])

In [42]:
oe

OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
                           ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                           ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1',
                            'IF']])

In [43]:
from sklearn.preprocessing import MinMaxScaler

In [44]:
scaler = MinMaxScaler()
scaler

MinMaxScaler()

In [45]:
from sklearn.compose import ColumnTransformer

In [46]:
ct = ColumnTransformer([('ordinal', oe, ['cut', 'color', 'clarity']),
                       ('scaler', scaler, ['carat', 'depth', 'table', 'x','y', 'z'])],
                      remainder = 'passthrough', verbose_feature_names_out=False).set_output(transform='pandas')
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinal',
                                 OrdinalEncoder(categories=[['Fair', 'Good',
                                                             'Very Good',
                                                             'Premium',
                                                             'Ideal'],
                                                            ['D', 'E', 'F', 'G',
                                                             'H', 'I', 'J'],
                                                            ['I1', 'SI2', 'SI1',
                                                             'VS2', 'VS1',
                                                             'VVS2', 'VVS1',
                                                             'IF']]),
                                 ['cut', 'color', 'clarity']),
                                ('scaler', MinMaxScaler(),
                                 ['carat', 'depth', 'table', 'x', 'y', 'z'])],
                  verbose_feature_names_out=False)

In [47]:
X_train_transform = ct.fit_transform(X_train)
X_train_transform

,cut,color,clarity,carat,depth,table,x,y,z
33544,4.0,0.0,3.0,0.031185,0.527778,0.230769,0.416201,0.076910,0.346154
26752,4.0,0.0,1.0,0.417879,0.527778,0.269231,0.778399,0.141087,0.642680
45914,4.0,1.0,3.0,0.068607,0.472222,0.269231,0.487896,0.089983,0.392060
3071,3.0,0.0,2.0,0.124740,0.519444,0.288462,0.554935,0.100679,0.455335
43829,4.0,1.0,6.0,0.043659,0.513889,0.250000,0.447858,0.081834,0.367246
...,...,...,...,...,...,...,...,...,...
11311,3.0,4.0,3.0,0.166320,0.483333,0.288462,0.606145,0.109677,0.486352
44869,4.0,3.0,2.0,0.089397,0.519444,0.211538,0.513966,0.094397,0.424318
38271,4.0,6.0,2.0,0.024948,0.522222,0.228846,0.408752,0.075042,0.337469
860,3.0,6.0,2.0,0.145530,0.550000,0.307692,0.570764,0.102377,0.473945


In [48]:
X_test_transform = ct.transform(X_test)

In [49]:
train_arr = X_train_transform.values
test_arr = X_test_transform.values
y_train_arr = y_train.values
k = 8
final_predict = []

In [50]:
from sklearn.metrics.pairwise import euclidean_distances

In [51]:
for i in range(len(test_arr)): # for memory management
    current_test_point = test_arr[i].reshape(1, -1) 
    
    distances = euclidean_distances(train_arr, current_test_point).flatten()
    nearest_indices = np.argsort(distances)[:k]
    predict_price = np.mean(y_train_arr[nearest_indices])
    final_predict.append(predict_price)

y_pred = np.array(final_predict)


In [52]:
from sklearn.metrics import mean_absolute_error, r2_score

In [53]:
r2_score(y_test, y_pred)

0.9512088282820864

In [55]:
mean_absolute_error(y_test,y_pred)

384.3825634352635

In [56]:
from sklearn.neighbors import KNeighborsRegressor

In [57]:
knn = KNeighborsRegressor(n_neighbors=8)
knn

KNeighborsRegressor(n_neighbors=8)

In [58]:
knn.fit(X_train_transform, y_train)

KNeighborsRegressor(n_neighbors=8)

In [59]:
y_pred1 = knn.predict(X_test_transform)
y_pred1

array([ 2748.5  ,  3992.75 ,  1846.375, ...,   906.75 , 14214.25 ,
        1073.625], shape=(10759,))

In [60]:
from sklearn.metrics import mean_absolute_error, r2_score

In [62]:
r2_score(y_test, y_pred1)

0.9512096211674899

In [61]:
mean_absolute_error(y_test, y_pred1)

384.36117436564734